In [14]:
import pandas as pd 
data = pd.read_csv('18-23数据.csv',encoding='utf-8')
data_yichang = pd.read_csv('异常件.csv',encoding='utf-8')
data_yichang['计划员']=0

data = data.append(data_yichang)

data['日期'] = pd.to_datetime(data['日期'], format='%Y%m%d').dt.strftime('%Y-%m-%d')

data = data[(data['日期']>='2023-01-01')&(data['日期']<'2023-07-01')]
data["month"] = pd.to_datetime(data['日期']).dt.month.astype(int)

sku_frequency = data.groupby(['零件号','month']).size().reset_index(name='sku_freq')
sku_frequency.head()

new_df = data.groupby(['零件号','日期','需求频次abc分类','计划员'])['需求数量'].sum().reset_index()
new_df['日期'] = pd.to_datetime(new_df['日期'])
new_df["month"] = pd.to_datetime(new_df['日期']).dt.month.astype(int)

new_df.head()

/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_28713/1093237813.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(data_yichang)


,零件号,日期,需求频次abc分类,计划员,需求数量,month
0,000979150EC,2023-02-24,F,4,1,2
1,026109150A,2023-01-04,F,4,2,1
2,02E301472,2023-01-04,C,2,2,1
3,02E301472,2023-01-27,C,2,3,1
4,02E301472,2023-01-28,C,2,5,1


In [15]:
key_info = new_df[['零件号','需求频次abc分类','计划员']].drop_duplicates()

In [16]:
import pandas as pd 
svg_pred = pd.read_csv('SVG预测.csv',encoding='utf-8')
yichang_data = pd.read_csv('异常件svg预测.csv',encoding='utf-8')
svg_pred = svg_pred.append(yichang_data)

svg_pred['日期'] = pd.to_datetime(svg_pred['日期'], format='%Y%m').dt.strftime('%Y-%m-%d')
svg_pred["month"] = pd.to_datetime(svg_pred['日期']).dt.month.astype(int)
svg_pred = svg_pred.rename(columns = {'零件代码':'零件号'})
svg_pred.head()

/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_28713/1794504682.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  svg_pred = svg_pred.append(yichang_data)


,零件号,日期,预测值,month
0,4B0501521E,2023-05-01,2.000000,5
1,533867910A,2023-03-01,0.302542,3
2,533867910A,2023-05-01,0.174583,5
3,5Q0133687C,2023-01-01,5.141276,1
4,5Q0133687C,2023-04-01,7.492224,4


In [17]:
real_demods = new_df.groupby(['零件号','month','需求频次abc分类','计划员'])['需求数量'].sum().reset_index(name='真实值')
real_demods.head()

,零件号,month,需求频次abc分类,计划员,真实值
0,000979150EC,2,F,4,1
1,026109150A,1,F,4,2
2,02E301472,1,C,2,10
3,02E301472,2,C,2,11
4,02E301472,3,C,2,22


In [18]:
sku_frequency.head()

,零件号,month,sku_freq
0,000979150EC,2,1
1,026109150A,1,1
2,02E301472,1,3
3,02E301472,2,4
4,02E301472,3,6


In [19]:
pred_real_demods = pd.merge(svg_pred,real_demods,on =['零件号','month'],how ='left')
pred_real_freq_demods = pd.merge(pred_real_demods,sku_frequency,on =['零件号','month'],how ='left')
pred_real_freq_demods['真实值'].fillna(0,inplace =True)
pred_real_freq_demods['sku_freq'].fillna(0,inplace =True)
pred_real_freq_demods.drop(['需求频次abc分类','计划员'],axis =1,inplace =True)
pred_real_freq_demods = pd.merge(pred_real_freq_demods,key_info,on = ['零件号'],how ='left')
pred_real_freq_demods.head()

,零件号,日期,预测值,month,真实值,sku_freq,需求频次abc分类,计划员
0,4B0501521E,2023-05-01,2.000000,5,2.0,2.0,E,4.0
1,533867910A,2023-03-01,0.302542,3,0.0,0.0,F,4.0
2,533867910A,2023-05-01,0.174583,5,1.0,1.0,F,4.0
3,5Q0133687C,2023-01-01,5.141276,1,8.0,4.0,C,4.0
4,5Q0133687C,2023-04-01,7.492224,4,2.0,2.0,C,4.0


In [7]:
# pred_real_freq_demods = pred_real_freq_demods[pred_real_freq_demods['真实值']!=0]

In [21]:
for i in ['预测值','真实值']:
    pred_real_freq_demods[i]= [1 if x ==0 else x for x in pred_real_freq_demods[i]]
pred_real_freq_demods['mape'] = (abs(pred_real_freq_demods['真实值'] - pred_real_freq_demods['预测值']) 
                                 / pred_real_freq_demods['真实值']) 
pred_real_freq_demods.head()
pred_real_freq_demods.to_csv('svg_pred_info.csv',encoding ='gb18030')

In [9]:
sku_mape = pred_real_freq_demods.groupby(['计划员','month'])['mape'].mean().reset_index()
pred_real_freq_demods['fwmape_values']=pred_real_freq_demods['mape']*pred_real_freq_demods['sku_freq']
fwmape_info = pred_real_freq_demods.groupby(['计划员','month'])['sku_freq','fwmape_values'].sum().reset_index()
fwmape_info['fwmape'] = fwmape_info['fwmape_values'] / fwmape_info['sku_freq']

wmape_info  = pred_real_freq_demods.groupby(['计划员','month'])['真实值','预测值'].sum().reset_index()
wmape_info['wmape'] = (abs(wmape_info['真实值'] - wmape_info['预测值']) / wmape_info['真实值'])

sku_tur_info = pd.merge(sku_mape,fwmape_info[['计划员','month','fwmape']],on = ['计划员','month'],how= 'left')
mape_thr_info = pd.merge(sku_tur_info,wmape_info[['计划员','month','wmape']],on = ['计划员','month'],how= 'left')

mape_thr_info.head()

/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_28713/3289026200.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  fwmape_info = pred_real_freq_demods.groupby(['计划员','month'])['sku_freq','fwmape_values'].sum().reset_index()
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_28713/3289026200.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  wmape_info  = pred_real_freq_demods.groupby(['计划员','month'])['真实值','预测值'].sum().reset_index()


,计划员,month,mape,fwmape,wmape
0,0.0,1,1.128485,0.372137,0.211721
1,0.0,2,0.781901,0.517167,0.413638
2,0.0,3,1.012866,0.300777,0.222807
3,0.0,4,0.359414,0.163725,0.035711
4,0.0,5,0.611607,0.109970,0.047544


In [10]:
pred_real_freq_demods.head(1)

,零件号,日期,预测值,month,真实值,sku_freq,需求频次abc分类,计划员,mape,fwmape_values
0,4B0501521E,2023-05-01,2.0,5,2.0,2.0,E,4.0,0.0,0.0


In [11]:
grouped_mape_info = pred_real_freq_demods.groupby(['month','需求频次abc分类'])['mape'].mean().reset_index()

grouped_wmape_info  = pred_real_freq_demods.groupby(['需求频次abc分类','month'])['真实值','预测值'].sum().reset_index()
grouped_wmape_info['wmape'] = (abs(grouped_wmape_info['真实值'] - grouped_wmape_info['预测值']) / grouped_wmape_info['真实值'])

grouped_fwmape_info = pred_real_freq_demods.groupby(['需求频次abc分类','month'])['sku_freq','fwmape_values'].sum().reset_index()
grouped_fwmape_info['fwmape'] = grouped_fwmape_info['fwmape_values'] / grouped_fwmape_info['sku_freq']


grouped_mape_tur_info = pd.merge(grouped_mape_info,grouped_wmape_info[['需求频次abc分类','month','wmape']],on = ['需求频次abc分类','month'],how= 'left')
group_mape_thr_info = pd.merge(grouped_mape_tur_info,grouped_fwmape_info[['需求频次abc分类','month','fwmape']],on = ['需求频次abc分类','month'],how= 'left')

group_mape_thr_info.head()


/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_28713/4053802085.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped_wmape_info  = pred_real_freq_demods.groupby(['需求频次abc分类','month'])['真实值','预测值'].sum().reset_index()
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_28713/4053802085.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped_fwmape_info = pred_real_freq_demods.groupby(['需求频次abc分类','month'])['sku_freq','fwmape_values'].sum().reset_index()


,month,需求频次abc分类,mape,wmape,fwmape
0,1,A,0.206046,0.228175,0.218258
1,1,B,1.082983,0.219050,0.470619
2,1,C,1.397016,0.207384,0.845971
3,1,D,1.401890,0.687208,1.009012
4,1,E,0.682517,0.194790,0.595938


In [12]:
# 计算mape，wmape 占比情况

# 定义区间
import matplotlib.pyplot as plt
import seaborn as sns

bins = [0, 0.10, 0.30, 0.50, 1.00, 100000]

# 使用cut函数将数据分成不同区间
pred_real_freq_demods['ScoreRange'] = pd.cut(pred_real_freq_demods['mape'], bins=bins, labels=['0-10%', '10%-30%', '30%-50%', '50%-100%', '>100%'],right =False)

pred_real_freq_demods['ScoreRange'].replace('NaN','0',inplace =True)
# 计算每个区间中的数据数量

grouped = pred_real_freq_demods.groupby(['month','ScoreRange']).size().reset_index(name='sku_mm_values')
# 计算每个月份内的总样本数
monthly_counts = pred_real_freq_demods.groupby(['month']).size().reset_index(name='mm_values')

# 合并统计结果
result = pd.merge(grouped, monthly_counts, on=['month'],how = 'left')

# 计算占比
result['Proportion'] = result['sku_mm_values'] / result['mm_values']
result = result[['month','ScoreRange','Proportion']]
result.tail(10)


/Users/kaka/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,month,ScoreRange,Proportion
20,5,0-10%,0.158349
21,5,10%-30%,0.233205
22,5,30%-50%,0.200576
23,5,50%-100%,0.312860
24,5,>100%,0.085413
25,6,0-10%,0.187873
26,6,10%-30%,0.227634
27,6,30%-50%,0.138171
28,6,50%-100%,0.312127
29,6,>100%,0.128231


In [13]:
import pandas as pd

excel_writer = pd.ExcelWriter('svg_pred_info_remove0.xlsx', engine='xlsxwriter')

# 将每个DataFrame写入不同的工作表
mape_thr_info.to_excel(excel_writer, sheet_name='sku_info', index=False)
group_mape_thr_info.to_excel(excel_writer, sheet_name='group_info', index=False)
result.to_excel(excel_writer, sheet_name='rate_info', index=False)

# 保存并关闭Excel Writer 对象
excel_writer.save()